# Predictiong the likely hood of Surviving in Titantic shipwreak
this notebook will use python-based Machine learning models to predict whether a passenger servivides the shipwreck or not.

## 1. Problem Definition:
> given a passenger features, can we predict wheter a passenger can survive the titanic shipwreck or not ? 

## 2. Data.
the original data came from kaggle look at for more info URL:https://www.kaggle.com/competitions/titanic/data
The data has been split into two groups:

training set (train.csv)
test set (test.csv)
The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.


## 3. Evalution
the evalution matrice for this problem is going to be accuracy: which is going to correct predicted fining of passengers.

## 4. Features:
Kaggle provides a data dictory detailing what each feature means, those can be found in the URL:
https://www.kaggle.com/competitions/titanic/data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### first lets discover our data and try to find some relastionships and even use confusion matrix to discover it.

In [2]:
df = pd.read_csv("./data/train.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### survivied is the class i am trying to predict in this case

let's compare sex and survivially of passengers.

In [4]:
df.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [5]:
df.groupby('Survived').Sex.value_counts()

Survived  Sex   
0         male      468
          female     81
1         female    233
          male      109
Name: Sex, dtype: int64

In [ ]:
countsSex = df.groupby('Survived')['Sex'].value_counts().unstack()

In [ ]:
countsSex

In [ ]:
fig,ax = plt.subplots(figsize = (10,10))
countsSex.plot(kind = "bar", stacked = True,ax = ax)
ax.set(title= "Surviabilty Vs Sex", xlabel = "survivability", ylabel ="Counts")
plt.legend(["Female", "Male"])
plt.show()

##### We can see that females are more likely to survive the titanicy sinking

### now let's fix our data and put all in numbers to train it into a model

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
df_tmp = df.copy()

lets fill up the data that is numberic with medium

In [10]:
# filling numbers 
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum(): # check for null values of numberic
            # add binary column which tells us whether that data actually missing
            df_tmp[label + "is_missing"] = pd.isnull(content)
            # fill missing numebric data 
            df_tmp[label] = content.fillna(content.median)

##### now if the data is not numberic we are going to fill with with its code but also we are going to add a codes for that NaN value

In [11]:
# fill non numberic data and put non numebric data into discrete data
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        # add a colum showing we have data missing in a certain column
        df_tmp[label+"is_missing"] = pd.isnull(content)
        # now put tat data into categorical order and add a 1 to it
        df_tmp[label] = pd.Categorical(content).codes+1

In [12]:
for label, content in df_tmp.items():
    if label.find("is_missing") != -1:
        df_tmp[label] = df_tmp[label].astype("category").cat.codes

In [9]:
### lets make a function that fixes our data to be used for train and test set

def preprocess_data(df):
    """
    this functions returns a dataframe that fixes the data.
    it will deal with missing values in our data and convert non-numeric values to be numeric
    """
    df_tmp = df.copy()
    
    if "Survived" in df_tmp.columns:
        df_tmp.drop("Survived",axis = 1 , inplace = True)
    # filling numbers 
    for label,content in df_tmp.items():
        if pd.api.types.is_numeric_dtype(content):
            
            if pd.isnull(content).sum(): # check for null values of numberic
                # add binary column which tells us whether that data actually missing
                df_tmp[label + "_is_missing"] = pd.isnull(content)
                # fill missing numebric data 
                df_tmp[label] = content.fillna(content.median())
            print(label)
        else:
            # add a colum showing we have data missing in a certain column
            df_tmp[label+"_is_missing"] = pd.isnull(content)
            # now put tat data into categorical order and add a 1 to it
            df_tmp[label] = pd.Categorical(content).codes+1
    
    for label, content in df_tmp.items():
        if label.find("_is_missing") != -1:
            df_tmp[label] = df_tmp[label].astype("category").cat.codes
    
    ## return the fixed dataFrame
    return df_tmp
    

In [14]:
df_tmp.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Ageis_missing',
       'Nameis_missing', 'Sexis_missing', 'Ticketis_missing',
       'Cabinis_missing', 'Embarkedis_missing'],
      dtype='object')

### now that we have got our data ready let's split it into test and traingset


In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train , y_test = train_test_split(df_tmp.drop("Survived",axis = 1),df_tmp["Survived"],test_size = 0.2)

now that we've got our data splited into train and test set lets try to put in it into our model

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [20]:
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

In [21]:
score= accuracy_score(y_test,y_pred)
score

0.8435754189944135

now lets use grid serach cv and make some hyperparameter tuning. 



In [22]:
%%time
from sklearn.model_selection import GridSearchCV
rf_grid = {
    "n_estimators":np.arange(10,200,10),
    "max_depth":[None,3,5,10],
    "min_samples_split": np.arange(2,10,2)
}
rc_model = GridSearchCV(RandomForestClassifier(n_jobs = -1,
                                              random_state = 42),param_grid = rf_grid,
                       cv = 5,
                       verbose = True)
rc_model.fit(x_train,y_train)

Fitting 5 folds for each of 304 candidates, totalling 1520 fits


KeyboardInterrupt: 

In [ ]:
rc_model.best_params_

In [ ]:
y_pred = rc_model.predict(x_test)
len(y_pred), len(y_test)

In [ ]:
score = accuracy_score(y_test,y_pred)
score

In [ ]:
%%time
### Now lets use RandsomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

rs_model = RandomizedSearchCV(RandomForestClassifier(n_jobs = -1,
                                                    random_state = 42),
                             param_distributions=rf_grid,
                             n_iter = 100,
                             cv = 5,
                             verbose = True)
# now fit our model 
rs_model.fit(x_train,y_train)

In [ ]:
accuracy_score(y_test,rs_model.predict(x_test))

### Fixing pre-process data function

In [147]:
### lets make a function that fixes our data to be used for train and test set

def preprocess_data(df):
    """
    this functions returns a dataframe that fixes the data.
    it will deal with missing values in our data and convert non-numeric values to be numeric
    """
    df_tmp = df.copy()
    
    if "Survived" in df_tmp.columns:
        df_tmp.drop("Survived",axis = 1 , inplace = True)
    # filling numbers 
    for label,content in df_tmp.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum(): # check for null values of numberic
                # add binary column which tells us whether that data actually missing
                df_tmp[label + "_is_missing"] = pd.isnull(content)
                # fill missing numebric data 
                df_tmp[label] = content.fillna(content.median())
            print(label)
        else:
            # add a colum showing we have data missing in a certain column
            if pd.isnull(content).sum():
                df_tmp[label+"_is_missing"] = pd.isnull(content)
            # now put tat data into categorical order and add a 1 to it
            df_tmp[label] = pd.Categorical(content).codes+1
    
    ## return the fixed dataFrame
    return df_tmp
    

In [153]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
x_train,y_train = preprocess_data(train),train["Survived"]
fixed_test = preprocess_data(test)

# modifiy fare_is_missing column manually
x_train["Fare_is_missing"] = False
fixed_test["Embarked_is_missing"] = False

fixed_test = fixed_test.reindex(columns=x_train.columns)
# merge labels and features of training set
x_train["Survived"] = y_train
fixed_train = x_train

PassengerId
Pclass
Age
SibSp
Parch
Fare
PassengerId
Pclass
Age
SibSp
Parch
Fare


In [149]:
np.setdiff1d(np.array(fixed_test.columns), np.intersect1d(np.array(x_train.columns),np.array(fixed_test.columns)))

array([], dtype=object)

In [110]:
x_train["Fare_is_missing"] = False
x_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_is_missing,Sex_is_missing,Age_is_missing,Ticket_is_missing,Cabin_is_missing,Embarked_is_missing,Fare_is_missing
0,1,3,109,2,22.0,1,0,524,7.2500,0,3,False,False,False,False,True,False,False
1,2,1,191,1,38.0,1,0,597,71.2833,82,1,False,False,False,False,False,False,False
2,3,3,354,1,26.0,0,0,670,7.9250,0,3,False,False,False,False,True,False,False
3,4,1,273,1,35.0,1,0,50,53.1000,56,3,False,False,False,False,False,False,False
4,5,3,16,2,35.0,0,0,473,8.0500,0,3,False,False,False,False,True,False,False


In [154]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          891 non-null    int64  
 1   Pclass               891 non-null    int64  
 2   Name                 891 non-null    int16  
 3   Sex                  891 non-null    int8   
 4   Age                  891 non-null    float64
 5   SibSp                891 non-null    int64  
 6   Parch                891 non-null    int64  
 7   Ticket               891 non-null    int16  
 8   Fare                 891 non-null    float64
 9   Cabin                891 non-null    int16  
 10  Embarked             891 non-null    int8   
 11  Age_is_missing       891 non-null    bool   
 12  Cabin_is_missing     891 non-null    bool   
 13  Embarked_is_missing  891 non-null    bool   
 14  Fare_is_missing      891 non-null    bool   
 15  Survived             891 non-null    int

In [155]:
fixed_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          418 non-null    int64  
 1   Pclass               418 non-null    int64  
 2   Name                 418 non-null    int16  
 3   Sex                  418 non-null    int8   
 4   Age                  418 non-null    float64
 5   SibSp                418 non-null    int64  
 6   Parch                418 non-null    int64  
 7   Ticket               418 non-null    int16  
 8   Fare                 418 non-null    float64
 9   Cabin                418 non-null    int8   
 10  Embarked             418 non-null    int8   
 11  Age_is_missing       418 non-null    bool   
 12  Cabin_is_missing     418 non-null    bool   
 13  Embarked_is_missing  418 non-null    bool   
 14  Fare_is_missing      418 non-null    bool   
dtypes: bool(4), float64(2), int16(2), int64(

### looking at our columns to feature engineer the data frame` 

In [143]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [145]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [98]:
fixed_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Name_is_missing',
       'Sex_is_missing', 'Age_is_missing', 'Ticket_is_missing',
       'Fare_is_missing', 'Cabin_is_missing', 'Embarked_is_missing'],
      dtype='object')

In [111]:
x_train["Survived"] = y_train
fixed_train = x_train

In [112]:
train.Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [165]:
fixed_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          418 non-null    int64  
 1   Pclass               418 non-null    int64  
 2   Name                 418 non-null    int16  
 3   Sex                  418 non-null    int8   
 4   Age                  418 non-null    float64
 5   SibSp                418 non-null    int64  
 6   Parch                418 non-null    int64  
 7   Ticket               418 non-null    int16  
 8   Fare                 418 non-null    float64
 9   Cabin                418 non-null    int8   
 10  Embarked             418 non-null    int8   
 11  Age_is_missing       418 non-null    bool   
 12  Cabin_is_missing     418 non-null    bool   
 13  Embarked_is_missing  418 non-null    bool   
 14  Fare_is_missing      418 non-null    bool   
dtypes: bool(4), float64(2), int16(2), int64(

In [166]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          891 non-null    int64  
 1   Pclass               891 non-null    int64  
 2   Name                 891 non-null    int16  
 3   Sex                  891 non-null    int8   
 4   Age                  891 non-null    float64
 5   SibSp                891 non-null    int64  
 6   Parch                891 non-null    int64  
 7   Ticket               891 non-null    int16  
 8   Fare                 891 non-null    float64
 9   Cabin                891 non-null    int16  
 10  Embarked             891 non-null    int8   
 11  Age_is_missing       891 non-null    bool   
 12  Cabin_is_missing     891 non-null    bool   
 13  Embarked_is_missing  891 non-null    bool   
 14  Fare_is_missing      891 non-null    bool   
dtypes: bool(4), float64(2), int16(3), int64(

In [135]:
col = fixed_test.pop("Fare_is_missing")


In [126]:
fixed_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Name_is_missing',
       'Sex_is_missing', 'Age_is_missing', 'Ticket_is_missing',
       'Cabin_is_missing', 'Embarked_is_missing'],
      dtype='object')

In [134]:
len(fixed_test.columns)

18

In [136]:
fixed_test.insert(len(fixed_test.columns),"Fare_is_missing",col)

In [137]:
fixed_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Name_is_missing',
       'Sex_is_missing', 'Age_is_missing', 'Ticket_is_missing',
       'Cabin_is_missing', 'Embarked_is_missing', 'Fare_is_missing'],
      dtype='object')

### lets define a function that finds the best paramters to be used in the model so we can submit our model


In [17]:
def get_model_best_model(data,parameters,n_iter,cv = 5):
    x_train,x_test,y_train, y_test = train_test_split(data.drop("Survived", axis =1), data["Survived"], test_size = 0.2)
    rs_model = RandomizedSearchCV(RandomForestClassifier(n_jobs = -1,
                                                    random_state = 42),
                             param_distributions=parameters,
                             n_iter = 100,
                             cv = cv,
                             verbose = True)
    # now fit our model 
    rs_model.fit(x_train,y_train)
    print("Our models Accuracy is {}".format(predict_and_evaluate_model(rs_model,x_test,y_test)))
    return rs_model
    

In [27]:
fixed_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Pclass_is_missing,Name_is_missing,Sex_is_missing,Age_is_missing,SibSp_is_missing,Parch_is_missing,Ticket_is_missing,Fare_is_missing,Cabin_is_missing,Embarked_is_missing
0,892,3,207,2,34.5,0,0,153,7.8292,0,...,0,0,0,0,0,0,0,0,1,0
1,893,3,404,1,47.0,1,0,222,7.0000,0,...,0,0,0,0,0,0,0,0,1,0
2,894,2,270,2,62.0,0,0,74,9.6875,0,...,0,0,0,0,0,0,0,0,1,0
3,895,3,409,2,27.0,0,0,148,8.6625,0,...,0,0,0,0,0,0,0,0,1,0
4,896,3,179,1,22.0,1,1,139,12.2875,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,354,2,27.0,0,0,268,8.0500,0,...,0,0,0,1,0,0,0,0,1,0
414,1306,1,284,1,39.0,0,0,325,108.9000,23,...,0,0,0,0,0,0,0,0,0,0
415,1307,3,333,2,38.5,0,0,347,7.2500,0,...,0,0,0,0,0,0,0,0,1,0
416,1308,3,385,2,27.0,0,0,221,8.0500,0,...,0,0,0,1,0,0,0,0,1,0


In [28]:
test.Fare.isna().sum()

1

In [29]:
train.Fare.isna().sum()

0

In [18]:
def predict_and_evaluate_model(model,x_test,y_test):
    return accuracy_score(y_test,model.predict(x_test))


In [36]:
%%time
from scipy.stats import randint
param_dist = {'n_estimators': randint(10, 1000,50),
              'max_depth': randint(1, 12,0),
              'min_samples_split': randint(2, 18,1),
              'min_samples_leaf': randint(2, 18,1),
              'max_features': ['auto', 'sqrt', 'log2', None]}

model = get_model_best_model(fixed_train,param_dist,n_iter=50)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Our models Accuracy is 0.8324022346368715
CPU times: total: 3min 53s
Wall time: 4min 41s


In [37]:
model.best_params_

{'max_depth': 6,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 4,
 'n_estimators': 478}

In [ ]:
fixed_train.drop("Survived", axis = 1).info()

In [ ]:
fixed_test.info()

In [41]:
opt_model = RandomForestClassifier(n_jobs = -1,
                                  n_estimators = 478,
                                   max_depth= 6,
                                  max_features = 'sqrt',
                                  min_samples_leaf=5,
                                  min_samples_split=4)
opt_model.fit(fixed_train.drop("Survived", axis = 1), fixed_train["Survived"])
opt_model_y_pred = opt_model.predict(fixed_test)

In [42]:
resulted_df = pd.DataFrame({"PassengerId":fixed_test["PassengerId"],
                           "Survived":opt_model_y_pred})

In [43]:
resulted_df.to_csv("Titanic_Survival_predctions.csv",index = False)

### Using XGBoost to optimize our model

In [114]:
import xgboost as xgb
from scipy.stats import randint

In [156]:
x_train,y_train = fixed_train.drop("Survived", axis = 1), fixed_train["Survived"]

In [197]:
# Define the hyperparameter search space
param_dist = {'n_estimators': np.arange(10,2000,10),
              'max_depth': [22],
              'learning_rate': [0.01],
              'gamma': [0],
              'subsample': [0.5, 0.7, 0.9, 1],
              'colsample_bytree': np.arange(0.5,1,0.01),
              'min_child_weight': [4]}

In [198]:
# Create an XGBoost classifier object
xgb_clf = xgb.XGBClassifier(random_state=42)

In [199]:
# Create a RandomizedSearchCV object
xgb_rs = RandomizedSearchCV(xgb_clf, param_distributions=param_dist, n_iter=200, cv=5, random_state=42,n_jobs = -1,verbose = True)

### Training using XGBoost

In [200]:
%%time
xgb_rs.fit(x_train,y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
CPU times: total: 7.5 s
Wall time: 5min 20s


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
       1330, 1340, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430,
       1440, 1450, 146

In [ ]:
model

In [201]:
# Print the best hyperparameters
print(xgb_rs.best_params_)

{'subsample': 1, 'n_estimators': 1000, 'min_child_weight': 4, 'max_depth': 22, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.7900000000000003}


In [202]:
y_pred_xgb = xgb_rs.predict(fixed_test)

In [203]:
y_pred_xgb

array([0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [163]:
y_pred_xgb

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

when looking at kaggle sumbssion requirment , we are reuired to add a DataFrame that has a `PassengerID` and `Survived` of the Titanic

In [204]:
resulted_df_xgb = pd.DataFrame({"PassengerId":fixed_test["PassengerId"],
                           "Survived":y_pred_xgb})
resulted_df_xgb.to_csv("./data/Titanic_Survival_predctions_xgb1.csv",index = False)

#### Best XGboost HyperParameters

In [192]:
best_paramxgb = {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 22, 'min_child_weight': 4, 'n_estimators': 666, 'subsample': 0.9}

### using My own Developed Neural Network for modeling

In [16]:
from Neural_Network_model import train_neural_network, model_predict,accuracy_score

In [17]:
x_train.shape[1]

22

In [18]:
net_shapes_dict = {
    "input_layer_shape" : (x_train.shape[1],1),
    "hidden_layer_shape" : (x_train.shape[1],16),
    "hidden_to_out_shape" : (1,16),
    "output_layer_shape" : (1,1)
    }

In [20]:
params = train_neural_network(net_shapes_dict,x_train,y_train,epoc = 100,alpha = 1,bias = 1) # optimal alpha and bias
# vanshing gradient problem 

### using our training set to get the accuracy 

# get predictedtion labels for train set
y_pred = model_predict(params,x_train)
# get accuracy score for train set
score = accuracy_score(y_train,y_pred)
print("Neural Network Accuracy Score {}% on Trainset".format(round(score*100,2)))

Neural Network Accuracy Score 61.62% on Trainset


In [26]:
max_score = 0
for i in np.arange(0.01,1,0.05):
    params = train_neural_network(net_shapes_dict,x_train,y_train,epoc = 1,alpha = i,bias = 1) # optimal alpha and bias
    # vanshing gradient problem 

    ### using our training set to get the accuracy 

    # get predictedtion labels for train set
    y_pred = model_predict(params,x_train)
    # get accuracy score for train set
    score = accuracy_score(y_train,y_pred)
    if score > max_score:
        max_score = score
    print("Neural Network Accuracy Max Score {}% on Trainset with alpha {}, ".format(round(score*100,2),i))

Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.01, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.060000000000000005, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.11, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.16000000000000003, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.21000000000000002, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.26, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.31000000000000005, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.36000000000000004, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.41000000000000003, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.46, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.51, 
Neural Network Accuracy Max Score 61.62% on Trainset with alpha 0.56, 
Neural Network Accuracy Max Score 61.62% on Trainset with

### Let's Use CatBoost

In [187]:
from catboost import CatBoostClassifier

# define model
model = CatBoostClassifier()

# define hyperparameters
hyperparameters = {
    "learning_rate": np.arange(0.01,1,0.05),
    "depth": np.arange(3,25,2),
    "l2_leaf_reg": np.arange(1, 9, 1),
    "iterations": np.arange(50, 501, 50),
    "random_strength": np.arange(0.1, 2.1, 0.1),
    "bagging_temperature": np.arange(0.5, 2.1, 0.5)
}
# Perform the hyperparameter search
clf = RandomizedSearchCV(model, param_distributions, n_iter=200, cv=5, random_state=42,verbose = True)




In [189]:
%%time
clf.fit(x_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
0:	learn: 0.6886414	total: 897us	remaining: 44ms
1:	learn: 0.6850607	total: 1.63ms	remaining: 39.1ms
2:	learn: 0.6814145	total: 2.19ms	remaining: 34.4ms
3:	learn: 0.6774639	total: 2.67ms	remaining: 30.7ms
4:	learn: 0.6732960	total: 3.18ms	remaining: 28.7ms
5:	learn: 0.6693076	total: 3.62ms	remaining: 26.6ms
6:	learn: 0.6656345	total: 4.1ms	remaining: 25.2ms
7:	learn: 0.6624056	total: 4.92ms	remaining: 25.8ms
8:	learn: 0.6586557	total: 5.69ms	remaining: 25.9ms
9:	learn: 0.6549571	total: 6.38ms	remaining: 25.5ms
10:	learn: 0.6514538	total: 6.94ms	remaining: 24.6ms
11:	learn: 0.6484261	total: 7.72ms	remaining: 24.5ms
12:	learn: 0.6455417	total: 8.25ms	remaining: 23.5ms
13:	learn: 0.6420365	total: 8.81ms	remaining: 22.7ms
14:	learn: 0.6389626	total: 9.32ms	remaining: 21.7ms
15:	learn: 0.6356578	total: 9.78ms	remaining: 20.8ms
16:	learn: 0.6326531	total: 10.3ms	remaining: 20ms
17:	learn: 0.6292408	total: 10.8ms	remaining: 19.1ms


0:	learn: 0.6548866	total: 552us	remaining: 27.1ms
1:	learn: 0.6229325	total: 1.4ms	remaining: 33.7ms
2:	learn: 0.5976350	total: 2.52ms	remaining: 39.5ms
3:	learn: 0.5721271	total: 3.21ms	remaining: 37ms
4:	learn: 0.5499646	total: 3.74ms	remaining: 33.7ms
5:	learn: 0.5309464	total: 4.19ms	remaining: 30.7ms
6:	learn: 0.5153958	total: 4.61ms	remaining: 28.3ms
7:	learn: 0.5035068	total: 5.09ms	remaining: 26.7ms
8:	learn: 0.4948519	total: 5.52ms	remaining: 25.1ms
9:	learn: 0.4878539	total: 5.95ms	remaining: 23.8ms
10:	learn: 0.4758263	total: 6.47ms	remaining: 23ms
11:	learn: 0.4689693	total: 6.96ms	remaining: 22ms
12:	learn: 0.4648692	total: 7.42ms	remaining: 21.1ms
13:	learn: 0.4577911	total: 7.85ms	remaining: 20.2ms
14:	learn: 0.4538746	total: 8.38ms	remaining: 19.6ms
15:	learn: 0.4470196	total: 8.89ms	remaining: 18.9ms
16:	learn: 0.4412184	total: 9.37ms	remaining: 18.2ms
17:	learn: 0.4363821	total: 9.87ms	remaining: 17.6ms
18:	learn: 0.4327905	total: 10.3ms	remaining: 16.9ms
19:	learn: 

0:	learn: 0.4702215	total: 1.14ms	remaining: 55.8ms
1:	learn: 0.4403273	total: 1.89ms	remaining: 45.4ms
2:	learn: 0.4272546	total: 2.63ms	remaining: 41.2ms
3:	learn: 0.4159231	total: 3.28ms	remaining: 37.8ms
4:	learn: 0.4032581	total: 3.81ms	remaining: 34.3ms
5:	learn: 0.3969088	total: 4.38ms	remaining: 32.2ms
6:	learn: 0.3936255	total: 5.09ms	remaining: 31.3ms
7:	learn: 0.3841624	total: 5.78ms	remaining: 30.4ms
8:	learn: 0.3772344	total: 7.43ms	remaining: 33.8ms
9:	learn: 0.3630829	total: 9.02ms	remaining: 36.1ms
10:	learn: 0.3588004	total: 9.69ms	remaining: 34.3ms
11:	learn: 0.3490136	total: 10.4ms	remaining: 32.9ms
12:	learn: 0.3414163	total: 11ms	remaining: 31.4ms
13:	learn: 0.3321372	total: 11.7ms	remaining: 30.2ms
14:	learn: 0.3234020	total: 12.4ms	remaining: 28.9ms
15:	learn: 0.3137353	total: 13ms	remaining: 27.6ms
16:	learn: 0.3010980	total: 13.7ms	remaining: 26.6ms
17:	learn: 0.2856657	total: 14.7ms	remaining: 26.1ms
18:	learn: 0.2780486	total: 15.5ms	remaining: 25.2ms
19:	lea

0:	learn: 0.6892598	total: 950us	remaining: 46.6ms
1:	learn: 0.6855362	total: 2.5ms	remaining: 59.9ms
2:	learn: 0.6819880	total: 5.75ms	remaining: 90.2ms
3:	learn: 0.6780926	total: 7.26ms	remaining: 83.5ms
4:	learn: 0.6740042	total: 8.63ms	remaining: 77.7ms
5:	learn: 0.6700891	total: 9.67ms	remaining: 70.9ms
6:	learn: 0.6665678	total: 11.8ms	remaining: 72.4ms
7:	learn: 0.6634364	total: 13.2ms	remaining: 69.3ms
8:	learn: 0.6606290	total: 14.1ms	remaining: 64.1ms
9:	learn: 0.6571233	total: 14.7ms	remaining: 58.9ms
10:	learn: 0.6536828	total: 15.4ms	remaining: 54.6ms
11:	learn: 0.6507134	total: 16ms	remaining: 50.6ms
12:	learn: 0.6476731	total: 16.6ms	remaining: 47.1ms
13:	learn: 0.6441967	total: 19.5ms	remaining: 50.1ms
14:	learn: 0.6411830	total: 24.1ms	remaining: 56.2ms
15:	learn: 0.6378522	total: 24.9ms	remaining: 52.9ms
16:	learn: 0.6349695	total: 25.9ms	remaining: 50.4ms
17:	learn: 0.6316587	total: 26.7ms	remaining: 47.5ms
18:	learn: 0.6288983	total: 27.4ms	remaining: 44.7ms
19:	lea

0:	learn: 0.6559845	total: 908us	remaining: 44.5ms
1:	learn: 0.6256237	total: 1.67ms	remaining: 40ms
2:	learn: 0.6010816	total: 2.1ms	remaining: 32.9ms
3:	learn: 0.5761940	total: 3.33ms	remaining: 38.4ms
4:	learn: 0.5545413	total: 5.55ms	remaining: 49.9ms
5:	learn: 0.5357788	total: 6.63ms	remaining: 48.6ms
6:	learn: 0.5208915	total: 8.57ms	remaining: 52.6ms
7:	learn: 0.5091749	total: 9.34ms	remaining: 49.1ms
8:	learn: 0.5006627	total: 10.2ms	remaining: 46.4ms
9:	learn: 0.4937549	total: 11.1ms	remaining: 44.6ms
10:	learn: 0.4820036	total: 12.7ms	remaining: 44.9ms
11:	learn: 0.4752246	total: 13.8ms	remaining: 43.8ms
12:	learn: 0.4714008	total: 14.8ms	remaining: 42.1ms
13:	learn: 0.4643096	total: 15.8ms	remaining: 40.6ms
14:	learn: 0.4604524	total: 17.4ms	remaining: 40.7ms
15:	learn: 0.4537117	total: 20.5ms	remaining: 43.6ms
16:	learn: 0.4479627	total: 22.2ms	remaining: 43.2ms
17:	learn: 0.4435038	total: 25.8ms	remaining: 45.9ms
18:	learn: 0.4399706	total: 27.2ms	remaining: 44.4ms
19:	lea

...

145:	learn: 0.2414005	total: 332ms	remaining: 123ms
146:	learn: 0.2401817	total: 335ms	remaining: 121ms
147:	learn: 0.2395264	total: 339ms	remaining: 119ms
148:	learn: 0.2388047	total: 344ms	remaining: 118ms
149:	learn: 0.2380256	total: 347ms	remaining: 116ms
150:	learn: 0.2372972	total: 352ms	remaining: 114ms
151:	learn: 0.2360986	total: 354ms	remaining: 112ms
152:	learn: 0.2351918	total: 359ms	remaining: 110ms
153:	learn: 0.2345395	total: 361ms	remaining: 108ms
154:	learn: 0.2332441	total: 363ms	remaining: 105ms
155:	learn: 0.2320836	total: 366ms	remaining: 103ms
156:	learn: 0.2316118	total: 370ms	remaining: 101ms
157:	learn: 0.2304938	total: 372ms	remaining: 98.9ms
158:	learn: 0.2296021	total: 375ms	remaining: 96.6ms
159:	learn: 0.2290284	total: 377ms	remaining: 94.2ms
160:	learn: 0.2270616	total: 379ms	remaining: 91.8ms
161:	learn: 0.2264919	total: 381ms	remaining: 89.3ms
162:	learn: 0.2255729	total: 384ms	remaining: 87.3ms
163:	learn: 0.2245517	total: 387ms	remaining: 84.9ms
164:	

198:	learn: 0.0023042	total: 398ms	remaining: 2ms
199:	learn: 0.0023041	total: 400ms	remaining: 0us
0:	learn: 0.4614393	total: 4.69ms	remaining: 934ms
1:	learn: 0.4176294	total: 6.55ms	remaining: 648ms
2:	learn: 0.4017734	total: 8.76ms	remaining: 575ms
3:	learn: 0.3935278	total: 10.6ms	remaining: 521ms
4:	learn: 0.3797299	total: 12.2ms	remaining: 474ms
5:	learn: 0.3626950	total: 14.5ms	remaining: 468ms
6:	learn: 0.3581218	total: 23.5ms	remaining: 648ms
7:	learn: 0.3373385	total: 30.6ms	remaining: 735ms
8:	learn: 0.3296146	total: 40.4ms	remaining: 858ms
9:	learn: 0.3107620	total: 46.5ms	remaining: 883ms
10:	learn: 0.3023918	total: 55.4ms	remaining: 952ms
11:	learn: 0.2875927	total: 60ms	remaining: 940ms
12:	learn: 0.2702767	total: 63ms	remaining: 906ms
13:	learn: 0.2589206	total: 69.1ms	remaining: 918ms
14:	learn: 0.2467776	total: 71.9ms	remaining: 887ms
15:	learn: 0.2359087	total: 73.8ms	remaining: 848ms
16:	learn: 0.2294278	total: 75.5ms	remaining: 813ms
17:	learn: 0.2195699	total: 77

0:	learn: 0.4577958	total: 1.93ms	remaining: 383ms
1:	learn: 0.4308723	total: 4.12ms	remaining: 408ms
2:	learn: 0.4162185	total: 5.75ms	remaining: 377ms
3:	learn: 0.4090875	total: 7.74ms	remaining: 379ms
4:	learn: 0.4039473	total: 9.47ms	remaining: 369ms
5:	learn: 0.3946342	total: 11.5ms	remaining: 373ms
6:	learn: 0.3833557	total: 14ms	remaining: 387ms
7:	learn: 0.3679104	total: 15.7ms	remaining: 378ms
8:	learn: 0.3567934	total: 17.4ms	remaining: 368ms
9:	learn: 0.3483293	total: 19.2ms	remaining: 366ms
10:	learn: 0.3399755	total: 21.1ms	remaining: 362ms
11:	learn: 0.3276975	total: 22.8ms	remaining: 357ms
12:	learn: 0.3200738	total: 24.4ms	remaining: 351ms
13:	learn: 0.3117388	total: 26.3ms	remaining: 349ms
14:	learn: 0.3051531	total: 28.8ms	remaining: 355ms
15:	learn: 0.2982665	total: 30.4ms	remaining: 350ms
16:	learn: 0.2901337	total: 32.1ms	remaining: 345ms
17:	learn: 0.2770926	total: 33.9ms	remaining: 342ms
18:	learn: 0.2668620	total: 36.4ms	remaining: 347ms
19:	learn: 0.2579568	tot

2:	learn: 0.3963279	total: 4.86ms	remaining: 319ms
3:	learn: 0.3911635	total: 6.97ms	remaining: 341ms
4:	learn: 0.3833908	total: 9.04ms	remaining: 353ms
5:	learn: 0.3727345	total: 10.9ms	remaining: 352ms
6:	learn: 0.3552172	total: 13.5ms	remaining: 373ms
7:	learn: 0.3371359	total: 15.2ms	remaining: 366ms
8:	learn: 0.3238804	total: 17.3ms	remaining: 368ms
9:	learn: 0.3131784	total: 21.2ms	remaining: 404ms
10:	learn: 0.3019473	total: 23.6ms	remaining: 406ms
11:	learn: 0.2945868	total: 26.6ms	remaining: 416ms
12:	learn: 0.2926026	total: 28.1ms	remaining: 405ms
13:	learn: 0.2823207	total: 29.6ms	remaining: 394ms
14:	learn: 0.2784126	total: 31.8ms	remaining: 392ms
15:	learn: 0.2675825	total: 33.6ms	remaining: 387ms
16:	learn: 0.2542881	total: 37.8ms	remaining: 407ms
17:	learn: 0.2442859	total: 39.6ms	remaining: 400ms
18:	learn: 0.2393181	total: 41.5ms	remaining: 395ms
19:	learn: 0.2324216	total: 44ms	remaining: 396ms
20:	learn: 0.2220033	total: 46ms	remaining: 392ms
21:	learn: 0.2155361	tot

KeyboardInterrupt: 

In [185]:
clf.best_params_

{'learning_rate': 0.1, 'l2_leaf_reg': 0.1, 'iterations': 200, 'depth': 7}

In [186]:
print("Training accuracy:", clf.score(x_train, y_train))

Training accuracy: 1.0


In [177]:
y_pred_cat_boost = clf.predict(np.array(fixed_test))
y_pred_cat_boost

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [180]:
### make a file to submit
resulted_df_catB = pd.DataFrame({"PassengerId":fixed_test["PassengerId"],
                           "Survived":y_pred_cat_boost})
resulted_df_catB.to_csv("./data/Titanic_Survival_predctions_catb.csv",index = False)